In [2]:
%pip install ascii-magic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.2/733.2 kB 5.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import customtkinter as ctk
from art import text2art
import ascii_magic
from PIL import Image, ImageTk
from tkinter import filedialog

# App Configuration
ctk.set_appearance_mode("light")

# Legible fonts list
LEGIBLE_FONTS = ["block", "caligraphy", "doom", "epic", "slant", "standard", "univers"]

class AsciiArtApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        # Color Palette
        self.bg_color = "#eaf6f6"  
        self.card_color = "#ffffff"  
        self.footer_color = "#2f6565"
        self.text_color = "#333333"

        # Window Configuration
        self.title("ASCII Art Generator")
        self.geometry("1000x700")
        self.minsize(800, 600)
        self.configure(bg=self.bg_color)

        # Grid Configuration
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(2, weight=1)

        # Header
        self.header = ctk.CTkLabel(
            self, 
            text="ASCII Art Generator", 
            font=("Courier", 28, "bold"),
            text_color=self.text_color
        )
        self.header.grid(row=0, column=0, pady=10)

        # Input Area
        self.input_frame = ctk.CTkFrame(self, fg_color=self.card_color)
        self.input_frame.grid(row=1, column=0, pady=10, padx=20, sticky="ew")
        
        self.input_label = ctk.CTkLabel(
            self.input_frame, 
            text="Type your text or object:", 
            font=("Helvetica", 16),
            text_color=self.text_color
        )
        self.input_label.grid(row=0, column=0, padx=10)
        
        self.input_field = ctk.CTkEntry(
            self.input_frame,
            width=400,
            height=40,
            fg_color="#ffffff",
            text_color=self.text_color,
            border_width=2,
            corner_radius=10
        )
        self.input_field.grid(row=0, column=1, padx=10)
        self.input_field.bind("<KeyRelease>", self.generate_ascii)
        
        # Font Selection Dropdown
        self.font_label = ctk.CTkLabel(
            self.input_frame,
            text="Select Font:",
            font=("Helvetica", 16),
            text_color=self.text_color
        )
        self.font_label.grid(row=0, column=2, padx=10)
        
        self.font_var = ctk.StringVar(value="block")
        self.font_dropdown = ctk.CTkComboBox(
            self.input_frame,
            values=LEGIBLE_FONTS,
            variable=self.font_var,
            command=self.generate_ascii
        )
        self.font_dropdown.grid(row=0, column=3, padx=10)

        # Image Upload Button
        self.upload_button = ctk.CTkButton(
            self.input_frame,
            text="Upload Image",
            command=self.upload_image
        )
        self.upload_button.grid(row=0, column=4, padx=10)

        # Output Area
        self.output_area = ctk.CTkTextbox(
            self,
            width=900,
            height=500,
            fg_color=self.card_color,
            text_color=self.text_color,
            corner_radius=10,
            font=("Courier", 14)
        )
        self.output_area.grid(row=2, column=0, pady=20, padx=20, sticky="nsew")
        self.output_area.configure(state="disabled")

        # Footer
        self.footer = ctk.CTkLabel(
            self,
            text="GWiST",
            font=("Helvetica", 14, "italic"),
            text_color="white",
            fg_color=self.footer_color,
            height=30
        )
        self.footer.grid(row=3, column=0, sticky="we")

    def generate_ascii(self, event=None):
        user_input = self.input_field.get().strip()
        selected_font = self.font_var.get()
        
        if user_input:
            try:
                ascii_art = text2art(user_input, font=selected_font)
            except:
                ascii_art = "Error generating ASCII art. Try a different font."
        else:
            ascii_art = ""
        
        self.output_area.configure(state="normal")
        self.output_area.delete("1.0", "end")
        self.output_area.insert("1.0", ascii_art)
        self.output_area.configure(state="disabled")

    def upload_image(self):
        try:
            file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
            if not file_path:
                return  # User canceled the dialog
                
            # Show a processing message
            self.output_area.configure(state="normal")
            self.output_area.delete("1.0", "end")
            self.output_area.insert("1.0", "Processing image... please wait...")
            self.output_area.update()  # Force update to show message
            
            # Open and validate the image first
            try:
                with Image.open(file_path) as img:
                    # Resize large images to prevent memory issues
                    max_size = (800, 800)
                    if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
                        img.thumbnail(max_size)
                        # Save temporary resized image
                        temp_path = file_path + "_resized.jpg"
                        img.save(temp_path)
                        file_path = temp_path
            except Exception as e:
                self.output_area.delete("1.0", "end")
                self.output_area.insert("1.0", f"Error opening image: {str(e)}")
                self.output_area.configure(state="disabled")
                return
                
            # Process with controlled parameters
            try:
                ascii_output = self.generate_ascii_image(file_path)
                self.output_area.delete("1.0", "end")
                self.output_area.insert("1.0", ascii_output)
            except Exception as e:
                self.output_area.delete("1.0", "end")
                self.output_area.insert("1.0", f"Error processing image: {str(e)}")
            finally:
                self.output_area.configure(state="disabled")
                # Remove temporary file if it was created
                if file_path.endswith("_resized.jpg"):
                    import os
                    try:
                        os.remove(file_path)
                    except:
                        pass
        except Exception as e:
            # Global exception handler to prevent crashes
            print(f"Critical error in upload_image: {str(e)}")
            self.output_area.configure(state="normal")
            self.output_area.delete("1.0", "end")
            self.output_area.insert("1.0", "An unexpected error occurred. Please try again with a different image.")
            self.output_area.configure(state="disabled")

    def generate_ascii_image(self, image_path):
        try:
            # Use very conservative settings to prevent crashes
            columns = 80  # Limit width
            width_ratio = 2.0  # Adjust for typical terminal/font aspect ratio
            
            output = ascii_magic.from_image_file(
                image_path,
                columns=columns,
                mode=ascii_magic.Modes.ASCII,
                width_ratio=width_ratio
            )
            
            # Convert to string safely
            return str(output)
        except Exception as e:
            return f"Error generating ASCII art: {str(e)}"

if __name__ == "__main__":
    app = AsciiArtApp()
    app.mainloop()


: 